# Update Notebook 1
2024-09-26

In [1]:
# For importing from folder without installing
import sys
sys.path.append('../src')

# Other packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Update 1: Working on nicer API
Can be installed as package (still have to update README.md for install). Inspired by [MNE](https://mne.tools/dev/index.html). 

Important classes:
- `Info` for details on measurement, like name, bias, and sampling rate.
- `Raw` for measurement sessions.
- `Events` for events found in `Raw`.

Important modules so far:
- `varv.preprocessing.eventdetection` is based on the work of Ruben Marchau. Code has been refactored and documented to be more usable.
- `varv.io.labview` for importing data from LabView .dat files.

> &#x1F3AF; **Goal**:
>
> Have everything in one place to make analysis easier down the road.

In [2]:
from varv.io import labview
from varv.preprocessing import eventdetection
from varv.base import Raw
from varv.events import Events

ModuleNotFoundError: No module named 'src'

Importing from LabView .dat and converting to .h5 (HDF) file format for easier loading. Note that the LabView reader uses `ast.literal_eval` rather than `eval` now which is a bit nicer and safer.

Set `CONVERT` to `True` in the cell below to read from the original .dat rather than .h5.

In [ ]:
CONVERT = True
fname = "240910_XC_tempV2_pore1_perf3_f0"
fname_carv = "240910_XC_tempV2_pore1_perf4_f0"

if CONVERT:
    folder = r"/Users/twh/Library/CloudStorage/OneDrive-DelftUniversityofTechnology/MSc/Year_2/NB5900_Master_End_Project/transfer_2875528_files_e7079156"
    raw = labview.read_measurement_dat(f"{os.path.join(folder, fname)}.dat", bamp=50, bfreq=200)
    raw_carv = labview.read_measurement_dat(f"{os.path.join(folder, fname_carv)}.dat")
    raw.save()
    raw_carv.save()
else:
    raw = Raw.from_h5(fname + '_raw')
    raw_carv = Raw.from_h5(fname_carv + '_raw')


In [ ]:
# Renaming for convenience
raw.info.name = "Variable Voltage"
raw_carv.info.name = "Constant Voltage"

# Smaller dataset to speed up things, comment out to test with full dataset
# raw.truncate(after=500000)
# raw_carv.truncate(after=500000)

In [ ]:
print(raw)

In [ ]:
print(raw_carv)

In [ ]:
fig, ax = raw.plot()
ax.set_ylim(-600, 900)

## Update 2: Quick look at the variable voltage data

In the variable voltage setup, a capacitive current can also flow in addition to the resistive flow. As visible in the plot below, the current distribution is much larger: from -600 to 800 pA. Note that the average current is still positive, as the bias voltage signal is purely positive (100 to 200 mV). The bias voltge is a triangle wave, which results in a square-wave like current trace due to the capacitance. This can be seen in the inlay below. Because in the square wave the current spends "most of its time" at the high or low, the total current plot looks like two (or actually several) current traces at different heights.

In [ ]:
fig, ax = raw.plot(start=2891920, stop=2952787)

# Inset Axes
x1, x2, y1, y2 = 2922353, 2922353 + 100, -600, 800  # subregion of the main plot
times = raw.get_time()
axins = ax.inset_axes(
    [0.7, 0.5, 0.47, 0.47],
    xlim=(times[x1], times[x2]), ylim=(y1, y2), xticklabels=[],)
    
raw.plot(start=x1, stop=x2, fig=fig, ax=axins, lowpass=None)
axins.set_xlabel(None)
axins.yaxis.set_label_position("right")
axins.yaxis.tick_right()
axins.set_title(None)
axins.get_legend().remove()

ax.indicate_inset_zoom(axins, edgecolor="black")
None

To make analysis of the data easier, we can filter out the frequency component related to the bias voltage (200 Hz) by applying a low-pass filter at 100 Hz. 

In [ ]:
raw.plot(start=2891920, stop=2952787, lowpass=100)

## Update 3: Current (open) state detection

Original method uses a Gaussian mixture model (GMM) to model the current values and find the distribution that corresponds tot the open pore current

In [ ]:
fig, ax = eventdetection.plot_open_state_fit(raw_carv, ignore_voltage=True)
ax.set_ylim(0, 250)


In [ ]:
raw_carv.info.sfreq

The standard settings do not work for variable voltage. The original algorithm expects an open state current between 220 and 250 pA. The open state current for variable voltage is lower as the mean current for the 100-200 mV triangle wave is 150 mV which is lower that the 180 mV DC bias in the constant method. As expected, results in an erroneous open state being found. Note that `ignore_voltage=True` here because the voltage state detection which is also used to eliminate currents in the open state finder also does not work for variable voltage. This can be resolved by changing the upper and lower bounds of this search space. Note that for variable voltage we first low-pass filter the signal to find the open state in the average voltage trace, hence the `bias_freq` parameter.

In [ ]:
eventdetection.find_open_state(raw, lower_bound=170, upper_bound=200, lowpass=100)
eventdetection.plot_open_state_fit(raw, degree=3, ignore_voltage=True, recalculate=False)

## Update 4: Voltage state detection

The original method for finding events used a GMM to find timepoints at which the voltage was off, scaled, or inverted. This is needed as events with an perturbed bias voltage should be thrown out as these are probably bad events like blockages in which the operator chose to interfere.

Three Guassians are fit to the data as shown below, note that the blue and the green Gaussians are really thin. Component p0 corresponds to a normal voltage, p1 to a scaled voltage, and p2 to an inverted votlage.

In [ ]:
raw_carv.reset_states()
eventdetection.plot_voltage_state(raw_carv, n_components=3)

This results in nicely classified data:

In [ ]:
eventdetection.find_open_state(raw_carv) # Also do the current labeling for a nicer plot.
eventdetection.find_bad_voltages(raw_carv)
raw_carv.plot('v')
fig, ax = raw_carv.plot()
ax.set_ylim(0, 250)

This is not the case for variable voltage data, as the GMM (not suprisingly) has trouble fitting a gaussian over the uniform voltage density from 100 to 200 mV.

In [ ]:
raw.reset_states()
eventdetection.plot_voltage_state(raw, n_components=3)

This is solved by providing one of the three distributions *a priori*, in this case a uniform distribution over the range from 90 to 210 mV.

In [ ]:
eventdetection.plot_voltage_state(raw, n_components=3, known_good_voltage=(90, 210))

As a result, the erroneous voltages are correctly labeled.

In [ ]:
raw.reset_states()
eventdetection.find_open_state(raw, 170, 200, lowpass=100)
eventdetection.find_bad_voltages(raw, known_good_voltage=(90, 210))
raw.plot('v')
raw.plot(lowpass=None)

## Update 5: Events

`Events` is a class created from `Raw`. `varv.preprocessing.eventdetection` runs under the hood to create the events.

In [ ]:
raw.reset_states()

In [ ]:
varv_settings = {
    "open_state_current": (170, 200),
    "lowpass": 100,
    "known_good_voltage": (90, 210),
}
events = Events.from_raw(raw, **varv_settings)

In [ ]:
print(f"Found {len(events)} events!")

In [ ]:
events.filter_by_event_length(verbose=True)

In [ ]:
events.properties

In [ ]:
events.properties.iloc[90]

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt, random

def plot_event(num):
    events.plot(num, lowpass=100)
    return ()
    
interact(plot_event, num=(1,len(events)-1,1));

In [ ]:
events.save("assets/240910_XC_tempV2_pore1_perf3_f0_eves.h5")